## Better!

In this notebook I will try refactoring the code to use the api more efficiently.


##### Load Libraries


In [274]:
import os
import json
import base64
from openai import OpenAI
from dotenv import load_dotenv

##### Load Environment Variables


In [275]:
load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

##### Data File Access


In [276]:
PATH_TO_JSON_FILE = "../data/visa_information.json"

try:
    with open(PATH_TO_JSON_FILE) as file:
        visa_information = json.load(file)
except:
    raise Exception("Failed to load json file.")

##### Adjust Data Structure For Easier Lookups


In [277]:
new_visa_information = {}

for origin, visa_info in visa_information.items():
    new_visa_information[origin] = {}
    """ 
        {
            origin: {}
        }
    """

    for destination, details in visa_info.get("visaFree", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaFree",
            "maxStay": details["maxStay"]
        }
    
    for destination, details in visa_info.get("visaOnArrival", {}).items():
        new_visa_information[origin][destination] = {
            "type": "visaOnArrival",
            "maxStay": details["maxStay"]
        }
    
    """
    {
        origin: 
        {
            destination: 
            {
                type: string,
                "maxStay": number    
            }    
        }
    }        
    """

##### Function To Get Visa Information

For the sake of scalability, we will use a dictionary with the name of the function and a reference to the function in the code, this way we can add functions easily to our system by appending to this dictionary.


In [278]:
async def get_visa_information(origin_country: str, destination_country: str, data = new_visa_information):    
    if not origin_country and not destination_country:
        raise ValueError("Both 'origin' and 'destination' countries are required.")
    
    visa_info = data.get(origin_country, {}).get(destination_country, {})
    
    return {
        'origin country': origin_country,
        'destination country': destination_country,
        'visa type': visa_info.get('type', 'visa required'),
        'maxStay': visa_info.get('maxStay', 0)
    }

In [279]:
# Add More Functions Here
function_calls_dict = {
    "get_visa_information": get_visa_information
}

##### Utility Functions


In [290]:

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def create_image_prompt(image_path):    
    base64_image = encode_image(image_path)

    return {
        "role": "user", 
        "content" :
            [
                {"type": "text", "text": "Here is a picture of my passport. Extract the abbreviation of the origin country from it."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},},
            ]
        }


##### Function Calling Service

This service class will be responsible for:

- Access to the functions dictionary.
- Extract tool calls from the assistant messages.
- Loop through the tool calls and call each function respectively.
- Append each tool call and its subsequent result into an array to be appended eventually to the main message stack.


In [280]:
class FunctionCallService():
    def __init__(self, functions):
        self.functions = functions
        self.tool_call_messages = []
        
    async def run(self, tool_call_prompt):

        for tool_call in tool_call_prompt.tool_calls:
            # Extract Function Name and Arguments
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            # Call The Function
            result = await self.__call_function(name, args)
            
            # In case our function returns an object
            if isinstance(result, object):
                result = json.dumps(result)
            
            self.tool_call_messages.append({
                "role": "tool",
                "tool_call_id":tool_call.id,
                "content": result
            })
            
        return self.tool_call_messages
            
    async def __call_function(self, name, args):
        if name in self.functions:
            func = self.functions[name]
            try:
                return await func(**args)
            except TypeError as e:
                return f"Error calling function '{name}': {str(e)}"
        else:
            return f"Error: Function '{name}' not found."
        
    def reset(self):
        self.function_call_messages = []

##### Chat Manager

The main idea behind having a manager for our chat is to allow easier management for the messages stack.
In this class we mainly run completions, and those completions – based on their type (tool calls, message etc..) – we route the flow of the messages.


In [281]:
from typing import List
from openai import OpenAI

class ChatManager():
    def __init__(self, tools, api_key: str ,model: str = "gpt-4o-mini", messages: List[object] = [] ):
        self.model = model
        self.tools = tools
        self.messages = messages
        self.client = OpenAI(api_key=api_key)
        self.functionCallService = FunctionCallService(function_calls_dict)
    
    async def next(self, message):
        self.messages.append(message)
        
        completion, is_function_calling = await self.__get_completion()
        
        # If there are function calling
        if is_function_calling:
            # Get the responses from the tools (Function Calls)
            tool_calls_responses = await self.functionCallService.run(completion)
            
            # Append the responses to the Messages Stack
            self.messages = self.messages + tool_calls_responses
            
            # Run a new completion with the new information
            completion, _ = await self.__get_completion()
            
        # Return the content of the prompt
        return completion.content
    
    async def __get_completion(self):
        response = client.chat.completions.create(
            model = self.model,
            tools = self.tools,
            messages = self.messages,
        )
        
        # Extract Message
        message = response.choices[0].message
        
        # Check if there is a function call
        is_function_calling = (
            hasattr(response.choices[0].message, "tool_calls") 
            and 
            message.tool_calls is not None)
        
        if not is_function_calling:
            self.messages.append({"role": "assistant", "content": message.content})
        else:
            self.messages.append(message)
        
        return message, is_function_calling
    
    def reset(self):
        self.messages = []
        self.functionCallService.reset()
    

##### Tools (Function Calls)


In [282]:
tools = [
    {
        "type": "function",
        "function": {  
        "name": "get_visa_information",
        "description": "Get Visa information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_country": {
                    "type": "string",
                    "description": "Abbreviation of the country from which the user's passport is issued",
                },
                "destination_country": {
                    "type": "string",
                    "description": "Abbreviation of the country the user is planning on visiting"
                }
            },
            "required": ["origin_country" ,"destination_country"],
            "additionalProperties": False
            },
            "strict": True
            },
        },
    ]

### Prompting


#### Prompt #1 - From Lebanon To Portugal - Text Prompts


In [268]:
# Create Manager
manager = ChatManager(tools=tools, api_key=api_key)

# Reset Manager
manager.reset()

# Create User Message
message = {"role": "user", "content": "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?" }

# Run Completion
completion = await manager.next(message)

print(completion)

[{'role': 'user', 'content': "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?"}]
Could you please tell me which country your passport is issued from? This will help me provide accurate visa information for your trip to Portugal.


In [269]:
# Provide Origin Country
origin_message = {"role": "user", "content": "My passport is from Lebanon."}

# Run Completion
completion = await manager.next(origin_message)

print(completion)

[{'role': 'user', 'content': "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?"}, {'role': 'assistant', 'content': 'Could you please tell me which country your passport is issued from? This will help me provide accurate visa information for your trip to Portugal.'}, {'role': 'user', 'content': 'My passport is from Lebanon.'}]
Tool Call ChatCompletionMessageToolCall(id='call_3tjnXOirtqUzBu6GIsCcgiaE', function=Function(arguments='{"origin_country":"LB","destination_country":"PT"}', name='get_visa_information'), type='function')
HERE [{'role': 'user', 'content': "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?"}, {'role': 'assistant', 'content': 'Could you please tell me which country your passport is issued from? This will help me provide accurate visa information for your trip to Portugal.'}, {'role': 'user', 'content': 'My passport is from Lebanon.'}, ChatCompletionMessage(content=None, refusal=None, role='assistan

#### Prompt #2 - From Netherlands To Portugal - Passport Image


In [291]:
# Create Manager
manager = ChatManager(tools=tools, api_key=api_key)

# Reset Manager
manager.reset()

# Create User Message
message = {"role": "user", "content": "Hello, I'm planning a trip to Portugal. Can you help me with the visa requirements?" }

# Run Completion
completion = await manager.next(message)

print(completion)

Sure! Could you please provide me with the country that your passport is issued from? This will help me find the visa requirements for your trip to Portugal.


In [292]:
IMAGE_PATH = "../data/passports/NLD Passport.jpg"

origin_message_image = create_image_prompt(IMAGE_PATH)

completion = await manager.next(origin_message_image)

print(completion)

As a passport holder from the Netherlands (NL), you do not need a visa to visit Portugal (PT) for short stays. You can stay for up to 90 days within a 180-day period for tourism or business purposes.

If you have any more questions about your trip, feel free to ask!


#### Prompt #3 - From Iceland to Singapore - Text Prompts


In [286]:
# Create Manager
manager = ChatManager(tools=tools, api_key=api_key)

# Reset in case older messages are present
manager.reset()

# Create User Message
message = {"role": "user", "content": "Hi, I need help determining the visa requirements to Singapore." }

# Run Completion
completion = await manager.next(message)

print(completion)

To assist you with the visa requirements for Singapore, could you please provide the following information:

1. Your passport's country of issuance (nationality).
2. The purpose of your visit (e.g., tourism, business, study, etc.). 

Once I have this information, I can provide you with the necessary visa requirements.


In [287]:
# Provide Origin Country
origin_message = {"role": "user", "content": "My passport is from Iceland."}

# Run Completion
completion = await manager.next(origin_message)

print(completion)

As a passport holder from Iceland, you will need a visa to visit Singapore. Unfortunately, you may not be able to stay for any duration without obtaining a visa first. 

Please check with the nearest Singaporean embassy or consulate for further details on the application process and any specific requirements you may need to fulfill.


#### Prompt #4 - From New Zealand To Portugal - Passport Image


In [293]:
# Create Manager
manager = ChatManager(tools=tools, api_key=api_key)

# Reset in case older messages are present
manager.reset()

# Create User Message
message = {"role": "user", "content": "Hi, I need help determining the visa requirements to Portugal." }

# Run Completion
completion = await manager.next(message)

print(completion)

Could you please provide me with the abbreviation of the country from which your passport is issued? This will help me determine the specific visa requirements for traveling to Portugal.


In [ ]:
IMAGE_PATH = "../data/passports/NZL Passport.png"

origin_message_image = create_image_prompt(IMAGE_PATH)

completion = await manager.next(origin_message_image)

print(completion)

As a passport holder from New Zealand (abbreviated as NZL), you will need a visa to travel to Portugal. However, the maximum stay details are not specified. It’s best to check with the Portuguese consulate or relevant authorities for specific visa types and conditions.


: 